In [1]:
import numpy as np
import pandas as pd
import copy
import time
import yfinance as yf
import datetime as dt
import plotly.express as px
from plotly.offline import plot
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import statsmodels.api as sm # linear regression
from stocktrends import Renko # 3rd-party library renko implementation

In [2]:
# atr function
def atr(df, n):
    df = df.copy()
    df['high-low'] = abs(df['High'] - df['Low'])
    df['high-pc'] = abs(df['High'] - df['Adj Close'].shift(1))
    df['low-pc'] = abs(df['Low'] - df['Adj Close'].shift(1))
    df['true-range'] = df[['high-low', 'high-pc', 'low-pc']].max(axis=1, skipna=False)
    df['atr'] = df['true-range'].rolling(n).mean()
    df.dropna(inplace=True)
    return df

In [3]:
# ordinary least square method => linear regression
def slope(ser, n):
    slopes = [i*0 for i in range(n-1)]
    for i in range(n,len(ser)+1):
        y = ser[i-n:i]
        x = np.array(range(n))
        y_scaled = (y - y.min())/(y.max() - y.min())
        x_scaled = (x - x.min())/(x.max() - x.min())
        x_scaled = sm.add_constant(x_scaled)
        model = sm.OLS(y_scaled,x_scaled)
        results = model.fit()
        slopes.append(results.params[-1])
    slope_angle = (np.rad2deg(np.arctan(np.array(slopes))))
    return np.array(slope_angle)

In [4]:
# renko function
def renko(df_original): # need original for atr function inside
    df = df_original.copy()
    df.reset_index(inplace=True) # convert index to column
    df = df.drop(['Close'], axis=1) # drop Close column
    df.columns  = ['date', 'open', 'high', 'low', 'close', 'volume'] # change column names
    renko_df = Renko(df)
    renko_df.brick_size = int(round(atr(df_original, 14)['atr'][-1], 0))
    renko_ohlc_df = renko_df.get_ohlc_data()
    renko_ohlc_df['bar-number'] = np.where(renko_ohlc_df['uptrend'] == True, 1, np.where(renko_ohlc_df['uptrend'] == False, -1, 0))
    for i in range(1, len(renko_ohlc_df['bar-number'])):
        if (renko_ohlc_df['bar-number'][i] > 0 and renko_ohlc_df['bar-number'][i-1] > 0):
            renko_ohlc_df['bar-number'][i] += renko_ohlc_df['bar-number'][i-1]
        elif (renko_ohlc_df['bar-number'][i] < 0 and renko_ohlc_df['bar-number'][i-1] < 0):
            renko_ohlc_df['bar-number'][i] += renko_ohlc_df['bar-number'][i-1]
    renko_ohlc_df.drop_duplicates(subset='date', keep='last', inplace=True) # if there is spike
    return renko_ohlc_df

In [5]:
# obv function
def obv(df):
    df = df.copy()
    df['daily-return'] = df['Adj Close'].pct_change()
    df['direction'] = np.where(df['daily-return'] > 0, 1, -1)
    df['direction'][0] = 0
    df['volume-direction'] = df['Volume'] * df['direction']
    df['obv'] = df['volume-direction'].cumsum()
    return df['obv']

In [6]:
# kpis
def cagr(df):
    df = df.copy()
    df['cumulative-return'] = (1 + df['return']).cumprod()
    n = len(df) / (252) # number of trading days per 1 year
    cagr = (df['cumulative-return'].tolist()[-1])**(1/n) - 1
    return cagr

def volatility(df):
    df = df.copy()
    annual_volatility = df['return'].std() * np.sqrt(252)
    return annual_volatility

def sharpe(df, rf):
    df = df.copy()
    sr = (cagr(df) - rf) / volatility(df)
    return sr

def maximum_drawdown(df):
    df = df.copy()
    df['cumulative-return'] = (1 + df['return']).cumprod() # value today
    df['cumulative-rolling-max'] = df['cumulative-return'].cummax()
    df['drawdown'] = df['cumulative-rolling-max'] - df['cumulative-return']
    df['drawdown-percent'] = df['drawdown'] / df['cumulative-rolling-max']
    m_dd = df['drawdown-percent'].max()
    return m_dd

def calmar_ratio(df):
    df = df.copy()
    c_r = cagr(df) / maximum_drawdown(df)
    return c_r

In [7]:
# time horizon set-up
start = dt.datetime.today() - dt.timedelta(365)
end = dt.datetime.today()

In [8]:
# NASDAQ most active by dollars stocks
# original_tickers = ['TSLA', 'AAPL', 'AMZN', 'QCOM', 'FB', 'NVDA'] # positive return stock only
original_tickers = ['TSLA', 'AAPL', 'AMZN', 'MSFT', 'QCOM', 'FB', 'AMD', 'NVDA', 'NFLX', 'ADBE']
original_ohlcvs = {}

In [9]:
# downloading data for all tickers
for ticker in original_tickers:
    original_ohlcvs[ticker] = yf.download(ticker, start, end, interval='1d')

# assigning new tickers
tickers = original_ohlcvs.keys()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [10]:
# basic data structure to implement back-testing
ohlcvs = copy.deepcopy(original_ohlcvs)
tickers_signal = {} # { 'MSFT': 'buy', 'AAPL': '', 'INTC': 'sell' }
tickers_signal_track = {}
tickers_return = {}

ohlc_renko = {}

In [11]:
# merging renko df with original ohlc df
for ticker in tickers:
    print(f'Merging for {ticker}')
    renko_df = renko(ohlcvs[ticker])
    renko_df.columns = ['date', 'open', 'high', 'low', 'close', 'uptrend', 'bar-number']
    ohlcvs[ticker]['date'] = ohlcvs[ticker].index
    ohlc_renko[ticker] = ohlcvs[ticker].merge(renko_df.loc[:, ['date', 'bar-number']], how = 'outer', on = 'date')
    ohlc_renko[ticker]['bar-number'].fillna(method = 'ffill', inplace=True)
    ohlc_renko[ticker]['obv']= obv(ohlc_renko[ticker])
    ohlc_renko[ticker]['obv_slope']= slope(ohlc_renko[ticker]['obv'], 5)
    tickers_signal[ticker] = ''
    tickers_return[ticker] = []
    tickers_signal_track[ticker] = []

Merging for TSLA


C:\Users\chond\anaconda3\envs\investment\lib\site-packages\stocktrends\indicators.py:58: FutureWarning:

Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.

<ipython-input-4-2f1a69976945>:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-4-2f1a69976945>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-5-e0ba58c6ba07>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

Merging for AAPL


C:\Users\chond\anaconda3\envs\investment\lib\site-packages\stocktrends\indicators.py:58: FutureWarning:

Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.

<ipython-input-4-2f1a69976945>:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-4-2f1a69976945>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-5-e0ba58c6ba07>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

Merging for AMZN


...

C:\Users\chond\anaconda3\envs\investment\lib\site-packages\stocktrends\indicators.py:58: FutureWarning:

Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.

<ipython-input-4-2f1a69976945>:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-4-2f1a69976945>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-5-e0ba58c6ba07>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

Merging for NFLX


C:\Users\chond\anaconda3\envs\investment\lib\site-packages\stocktrends\indicators.py:58: FutureWarning:

Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.

<ipython-input-4-2f1a69976945>:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-4-2f1a69976945>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-5-e0ba58c6ba07>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

Merging for ADBE


C:\Users\chond\anaconda3\envs\investment\lib\site-packages\stocktrends\indicators.py:58: FutureWarning:

Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.

<ipython-input-4-2f1a69976945>:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-4-2f1a69976945>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-5-e0ba58c6ba07>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

In [12]:
# identifying signals and calculating return and incorporate stop-loss accordingly
obv_slope_threshold = 15
for ticker in tickers:
    print(f'Calculating returns for {ticker}')
    for i in range(len(ohlc_renko[ticker])):
        
        if (tickers_signal[ticker] == ''): 
            tickers_return[ticker].append(0) # no signal equals no return for that candle   
            
            if (ohlc_renko[ticker]['bar-number'][i] >= 2 and ohlc_renko[ticker]['obv_slope'][i] > obv_slope_threshold):
                tickers_signal[ticker] = 'Buy'
                tickers_signal_track[ticker].append('Start Buy')
            elif (ohlc_renko[ticker]['bar-number'][i] <= -2 and ohlc_renko[ticker]['obv_slope'][i] < -obv_slope_threshold):
                tickers_signal[ticker] = 'Sell'
                tickers_signal_track[ticker].append('Start Sell')
            else:
                tickers_signal_track[ticker].append('dnt')
        
        elif (tickers_signal[ticker] == 'Buy'):
            tickers_return[ticker].append((ohlc_renko[ticker]['Adj Close'][i] / ohlc_renko[ticker]['Adj Close'][i-1]) - 1)
            tickers_signal_track[ticker].append('Continue')
            if (ohlc_renko[ticker]['bar-number'][i] <= -2 and ohlc_renko[ticker]['obv_slope'][i] < -obv_slope_threshold):
                tickers_signal[ticker] = 'Sell'
                tickers_signal_track[ticker].pop()
                tickers_signal_track[ticker].append('Start Sell Reversal')
            elif (ohlc_renko[ticker]['bar-number'][i] < 2):
                tickers_signal[ticker] = '' # change from buy signal to nth to close off
                tickers_signal_track[ticker].pop()
                tickers_signal_track[ticker].append('Close Buy')
                
        elif (tickers_signal[ticker] == 'Sell'):
            tickers_return[ticker].append((ohlc_renko[ticker]['Adj Close'][i-1] / ohlc_renko[ticker]['Adj Close'][i]) - 1)
            tickers_signal_track[ticker].append('Continue')
            if (ohlc_renko[ticker]['bar-number'][i] >= 2 and ohlc_renko[ticker]['obv_slope'][i] > obv_slope_threshold):
                tickers_signal[ticker] = 'Buy'
                tickers_signal_track[ticker].pop()
                tickers_signal_track[ticker].append('Start Buy Reversal')
            elif (ohlc_renko[ticker]['bar-number'][i] >- 2):
                tickers_signal[ticker] = '' # change from sell to nth to close off
                tickers_signal_track[ticker].pop()
                tickers_signal_track[ticker].append('Close Sell')
                
    ohlc_renko[ticker]['return'] = np.array(tickers_return[ticker])
    ohlc_renko[ticker]['signal'] = np.array(tickers_signal_track[ticker])

Calculating returns for TSLA
Calculating returns for AAPL
Calculating returns for AMZN
Calculating returns for MSFT
Calculating returns for QCOM
Calculating returns for FB
Calculating returns for AMD
Calculating returns for NVDA
Calculating returns for NFLX
Calculating returns for ADBE


In [13]:
ohlc_renko['TSLA']

,Open,High,Low,Close,Adj Close,Volume,date,bar-number,obv,obv_slope,return,signal
0,72.510002,76.722000,72.500000,76.300003,76.300003,90871000,2019-12-16,1.0,0,0.000000,0.000000,dnt
1,75.797997,77.099998,75.180000,75.797997,75.797997,42484000,2019-12-17,1.0,-42484000,0.000000,0.000000,dnt
2,76.125999,79.043999,76.115997,78.629997,78.629997,70605000,2019-12-18,2.0,28121000,0.000000,0.000000,dnt
3,79.463997,81.370003,79.300003,80.807999,80.807999,90535500,2019-12-19,2.0,118656500,0.000000,0.000000,dnt
4,82.057999,82.599998,80.038002,81.117996,81.117996,73763500,2019-12-20,2.0,192420000,42.913862,0.000000,Start Buy
...,...,...,...,...,...,...,...,...,...,...,...,...
246,604.919983,648.789978,603.049988,641.760010,641.760010,56309700,2020-12-07,6.0,2885807000,35.994923,0.071314,Continue
247,625.510010,651.280029,618.500000,649.880005,649.880005,64265000,2020-12-08,6.0,2950072000,44.364956,0.012653,Continue
248,653.690002,654.320007,588.000000,604.479980,604.479980,71291200,2020-12-09,6.0,2878780800,36.549666,-0.069859,Continue
249,574.369995,627.750000,566.340027,627.070007,627.070007,67083200,2020-12-10,6.0,2945864000,36.824817,0.037371,Continue


In [23]:
# create figure
signal_ticker = 'NVDA'

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=ohlc_renko[signal_ticker].index, y=ohlc_renko[signal_ticker]['Adj Close'], name='Price'))

for i in range(len(ohlc_renko[signal_ticker])):
    if (ohlc_renko[signal_ticker]['signal'][i] == 'Start Buy'):
        fig.add_trace(go.Scatter(x=[ohlc_renko[signal_ticker].index[i], ohlc_renko[signal_ticker].index[i]], y=[0,1], name='Start Buy', marker_color='rgba(0, 255, 0, 1)'), secondary_y=True)
    if (ohlc_renko[signal_ticker]['signal'][i] == 'Close Buy'):
        fig.add_trace(go.Scatter(x=[ohlc_renko[signal_ticker].index[i], ohlc_renko[signal_ticker].index[i]], y=[0,1], name='Close Buy', marker_color='rgba(0, 255, 0, 0.1)'), secondary_y=True)
    if (ohlc_renko[signal_ticker]['signal'][i] == 'Start Sell Reversal'):
        fig.add_trace(go.Scatter(x=[ohlc_renko[signal_ticker].index[i], ohlc_renko[signal_ticker].index[i]], y=[0,1], name='Start Sell Reversal', marker_color='rgba(255, 255, 0, 1)'), secondary_y=True)
    if (ohlc_renko[signal_ticker]['signal'][i] == 'Start Sell'):
        fig.add_trace(go.Scatter(x=[ohlc_renko[signal_ticker].index[i], ohlc_renko[signal_ticker].index[i]], y=[0,1], name='Start Sell', marker_color='rgba(255, 0, 0, 1)'), secondary_y=True)
    if (ohlc_renko[signal_ticker]['signal'][i] == 'Close Sell'):
        fig.add_trace(go.Scatter(x=[ohlc_renko[signal_ticker].index[i], ohlc_renko[signal_ticker].index[i]], y=[0,1], name='Close Sell', marker_color='rgba(255, 0, 0, 0.1)'), secondary_y=True)
    if (ohlc_renko[signal_ticker]['signal'][i] == 'Start Buy Reversal'):
        fig.add_trace(go.Scatter(x=[ohlc_renko[signal_ticker].index[i], ohlc_renko[signal_ticker].index[i]], y=[0,1], name='Start Buy Reversal', marker_color='rgba(0, 255, 255, 1)'), secondary_y=True)

# format figure
fig.update_layout(title_text=f'S2 Signals Backtested {dt.datetime.today()} Price of {signal_ticker}')
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Price')
fig.update_yaxes(title_text='Reference', secondary_y=True)

In [24]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=ohlc_renko[signal_ticker].index, y=ohlc_renko[signal_ticker]['bar-number'], name='Bar Number'))

for i in range(len(ohlc_renko[signal_ticker])):
    if (ohlc_renko[signal_ticker]['signal'][i] == 'Start Buy'):
        fig.add_trace(go.Scatter(x=[ohlc_renko[signal_ticker].index[i], ohlc_renko[signal_ticker].index[i]], y=[0,1], name='Start Buy', marker_color='rgba(0, 255, 0, 1)'), secondary_y=True)
    if (ohlc_renko[signal_ticker]['signal'][i] == 'Close Buy'):
        fig.add_trace(go.Scatter(x=[ohlc_renko[signal_ticker].index[i], ohlc_renko[signal_ticker].index[i]], y=[0,1], name='Close Buy', marker_color='rgba(0, 255, 0, 0.1)'), secondary_y=True)
    if (ohlc_renko[signal_ticker]['signal'][i] == 'Start Sell Reversal'):
        fig.add_trace(go.Scatter(x=[ohlc_renko[signal_ticker].index[i], ohlc_renko[signal_ticker].index[i]], y=[0,1], name='Start Sell Reversal', marker_color='rgba(255, 255, 0, 1)'), secondary_y=True)
    if (ohlc_renko[signal_ticker]['signal'][i] == 'Start Sell'):
        fig.add_trace(go.Scatter(x=[ohlc_renko[signal_ticker].index[i], ohlc_renko[signal_ticker].index[i]], y=[0,1], name='Start Sell', marker_color='rgba(255, 0, 0, 1)'), secondary_y=True)
    if (ohlc_renko[signal_ticker]['signal'][i] == 'Close Sell'):
        fig.add_trace(go.Scatter(x=[ohlc_renko[signal_ticker].index[i], ohlc_renko[signal_ticker].index[i]], y=[0,1], name='Close Sell', marker_color='rgba(255, 0, 0, 0.1)'), secondary_y=True)
    if (ohlc_renko[signal_ticker]['signal'][i] == 'Start Buy Reversal'):
        fig.add_trace(go.Scatter(x=[ohlc_renko[signal_ticker].index[i], ohlc_renko[signal_ticker].index[i]], y=[0,1], name='Start Buy Reversal', marker_color='rgba(0, 255, 255, 1)'), secondary_y=True)

# format figure
fig.update_layout(title_text=f'S2 Signals Backtested {dt.datetime.today()} Renko of {signal_ticker}')
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Renko Blocks')
fig.update_yaxes(title_text='Reference', secondary_y=True)

In [25]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=ohlc_renko[signal_ticker].index, y=ohlc_renko[signal_ticker]['obv_slope'], name='Bar Number'))

for i in range(len(ohlc_renko[signal_ticker])):
    if (ohlc_renko[signal_ticker]['signal'][i] == 'Start Buy'):
        fig.add_trace(go.Scatter(x=[ohlc_renko[signal_ticker].index[i], ohlc_renko[signal_ticker].index[i]], y=[0,1], name='Start Buy', marker_color='rgba(0, 255, 0, 1)'), secondary_y=True)
    if (ohlc_renko[signal_ticker]['signal'][i] == 'Close Buy'):
        fig.add_trace(go.Scatter(x=[ohlc_renko[signal_ticker].index[i], ohlc_renko[signal_ticker].index[i]], y=[0,1], name='Close Buy', marker_color='rgba(0, 255, 0, 0.1)'), secondary_y=True)
    if (ohlc_renko[signal_ticker]['signal'][i] == 'Start Sell Reversal'):
        fig.add_trace(go.Scatter(x=[ohlc_renko[signal_ticker].index[i], ohlc_renko[signal_ticker].index[i]], y=[0,1], name='Start Sell Reversal', marker_color='rgba(255, 255, 0, 1)'), secondary_y=True)
    if (ohlc_renko[signal_ticker]['signal'][i] == 'Start Sell'):
        fig.add_trace(go.Scatter(x=[ohlc_renko[signal_ticker].index[i], ohlc_renko[signal_ticker].index[i]], y=[0,1], name='Start Sell', marker_color='rgba(255, 0, 0, 1)'), secondary_y=True)
    if (ohlc_renko[signal_ticker]['signal'][i] == 'Close Sell'):
        fig.add_trace(go.Scatter(x=[ohlc_renko[signal_ticker].index[i], ohlc_renko[signal_ticker].index[i]], y=[0,1], name='Close Sell', marker_color='rgba(255, 0, 0, 0.1)'), secondary_y=True)
    if (ohlc_renko[signal_ticker]['signal'][i] == 'Start Buy Reversal'):
        fig.add_trace(go.Scatter(x=[ohlc_renko[signal_ticker].index[i], ohlc_renko[signal_ticker].index[i]], y=[0,1], name='Start Buy Reversal', marker_color='rgba(0, 255, 255, 1)'), secondary_y=True)

# format figure
fig.update_layout(title_text=f'S2 Signals Backtested {dt.datetime.today()} OBV Slope of {signal_ticker}')
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='OBV Slope')
fig.update_yaxes(title_text='Reference', secondary_y=True)

In [17]:
# calculating overall strategy's KPIs
strategy = pd.DataFrame()
for ticker in tickers:
    strategy[ticker] = ohlc_renko[ticker]['return']
strategy['return'] = strategy.mean(axis=1) # assume equal capital allocation
strategy['cumulative-return'] = (1 + strategy['return']).cumprod()
strategy_cagr = cagr(strategy)
strategy_volatility = volatility(strategy)
strategy_sharpe = sharpe(strategy, 0.01)
strategy_max_dd = maximum_drawdown(strategy)
strategy_calmar_ratio = calmar_ratio(strategy)

In [18]:
strategy

,TSLA,AAPL,AMZN,MSFT,QCOM,FB,AMD,NVDA,NFLX,ADBE,return,cumulative-return
0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,1.000000
1,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,1.000000
2,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,1.000000
3,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,1.000000
4,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
246,0.071314,0.012270,0.0,0.0,0.002538,0.0,0.000319,-0.003564,0.0,0.000000,0.008288,1.297272
247,0.012653,0.005091,0.0,0.0,0.004936,0.0,-0.012225,0.019232,0.0,0.000000,0.002969,1.301123
248,-0.069859,-0.020904,0.0,0.0,-0.017506,0.0,-0.033254,0.032423,0.0,-0.023300,-0.013240,1.283896
249,0.037371,0.011989,0.0,0.0,-0.001731,0.0,0.020372,-0.003199,0.0,-0.014202,0.005060,1.290393


In [19]:
print(f'Strategy CAGR: {strategy_cagr}')
print(f'Strategy Volatility: {strategy_volatility}')
print(f'Strategy Sharpe: {strategy_sharpe}')
print(f'Strategy Maximum Drawdown: {strategy_max_dd}')
print(f'Strategy Calmar Ratio: {strategy_calmar_ratio}')

Strategy CAGR: 0.27706443939039116
Strategy Volatility: 0.28733371669071023
Strategy Sharpe: 0.9294573656939217
Strategy Maximum Drawdown: 0.212263586922808
Strategy Calmar Ratio: 1.3052848272612518


In [20]:
# KPIs for each stocks
individual_cagr = {}
individual_volatility = {}
individual_sharpe = {}
individual_max_dd = {}
individual_calmar_ratio = {}

for ticker in tickers:
    print(f'Calculating KPIs for {ticker}')
    individual_cagr[ticker] = cagr(ohlc_renko[ticker])
    individual_volatility[ticker] = volatility(ohlc_renko[ticker])
    individual_sharpe[ticker] = sharpe(ohlc_renko[ticker], 0.01)
    individual_max_dd[ticker] = maximum_drawdown(ohlc_renko[ticker])
    individual_calmar_ratio[ticker] = calmar_ratio(ohlc_renko[ticker])

Calculating KPIs for TSLA
Calculating KPIs for AAPL
Calculating KPIs for AMZN
Calculating KPIs for MSFT
Calculating KPIs for QCOM
Calculating KPIs for FB
Calculating KPIs for AMD
Calculating KPIs for NVDA
Calculating KPIs for NFLX
Calculating KPIs for ADBE


In [21]:
individual_cagr

{'TSLA': 0.3216139179371156,
 'AAPL': 0.6303310569299432,
 'AMZN': 0.501563314498267,
 'MSFT': -0.06827617275680709,
 'QCOM': 0.42313793620816864,
 'FB': 0.1659663488464178,
 'AMD': -0.033962453484648325,
 'NVDA': 1.3314329101243692,
 'NFLX': -0.1533521779214262,
 'ADBE': -0.4220211335122527}

In [22]:
# create figure
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=strategy.index, y=strategy['cumulative-return'], name='Cumulative Return'))

# format figure
fig.update_layout(title_text=f'Cumulative Return of Portfolio of S2 - Renko OBV Backtested {dt.datetime.today()}')
fig.update_xaxes(title_text='Period')
fig.update_yaxes(title_text='Return')